In [91]:
import random
import numpy as np
from collections import deque

#### Parámetros

In [92]:
# Ejemplo del enunciado
# T = 24*60 # 24h en minutos
T = 1*24*60 #un mes en minutos #pongo 1 dia

nPistas = 3
nCoches = 20
nPuertas = 50

# lambda0 = getlambda0(t)
mu11 = 10.007615178514001
sigma11 = 3.0377002745229005
a12 = 10
b12 = 15
lambda2 = 1 / 2
lambda3 = 0.1

In [93]:
def getlambda0(t):
    # t está en minutos, convertir a horas:
    t = t/60
    t = t%24 ##########ojo esto##########
    # Intervalo de 0 a 5 horas, lineal de 5 a 7
    if 0 <= t < 5:
        return 5 + (7 - 5) * (t - 0) / (5 - 0)
    # Intervalo de 5 a 8 horas, lineal de 7 a 6
    elif 5 <= t < 8:
        return 7 + (6 - 7) * (t - 5) / (8 - 5)
    # Intervalo de 8 a 15 horas, lineal de 6 a 9
    elif 8 <= t < 15:
        return 6 + (9 - 6) * (t - 8) / (15 - 8)
    # Intervalo de 15 a 17 horas, lineal de 9 a 6
    elif 15 <= t < 17:
        return 9 + (6 - 9) * (t - 15) / (17 - 15)
    # Intervalo de 17 a 24 horas, lineal de 6 a 5
    elif 14 <= t <= 24:
        return 6 + (5 - 6) * (t - 17) / (24 - 17)
    # Si el tiempo está fuera del rango de 0 a 24 horas
    else:
        return 10E9  

#### Funciones auxiliares para generación de tiempos

In [94]:
# Funciones auxiliares generacion de tiempos
def t_llegada(t):
    return random.expovariate(getlambda0(t))*60 #nº muy pequeños, tiene sentido?

def t_aterrizaje():
    # return np.random.normal(mu11, sigma11)
    # return random.normalvariate(mu11, sigma11) 
    while True:  # Bucle infinito hasta obtener un número válido
        resultado = np.random.normal(mu11, sigma11)
        if resultado > 0:  # Comprobar si el resultado es mayor que cero
            return resultado
        
def t_traslado(): #N1- N2
    return np.random.exponential(1/lambda2)

def t_desembarques():
    return np.random.exponential(1/lambda3)

def t_salidas():
    return np.random.uniform(a12, b12)

#### Variables

In [95]:
t = 0                   # Variable reloj. tiempo asíncrono.
n1 = n2 = n3 = 0        # Variables de estado nº clientes
NLL1 = NLL2 = NLL3 = 0  # Variables contador nº llegadas
NS1 = NS2 = NS3 = 0     # Variables contador nº salidas

# Lista de tiempos de eventos
LISTA = {
    'tLL1': 10E9,   
    'tS1':  [10E9]*nPistas,
    'tS2':  [10E9]*nCoches,
    'tS3':  [10E9]*nPuertas,
    'tSS':  [10E9]*nPistas
}

# Arrays for arrival and departure times
LL1 = []
LL2 = []
LL3 = []
S1 = []
S2 = []
S3 = []

n_med_n1 = n_med_n2 = n_med_n3 = 0 #nº medio de pistas ocupadas
nmed_pistasOcupadas = 0

n_cola_aterrizaje = 0 # Contador de aviones esperando para aterrizar
pistasOcupadas = 0

esperando_coche = 0

#### rutina aterrizaje

In [96]:

def rutina_aterrizaje(t_suc):
    
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, esperando_coche, pistasOcupadas,n_cola_aterrizaje
    global esperando_a_aterrizar, esperando_puerta
    
    print(f'----Llega un vuelo en el minuto {t_suc}----')
    # para luego calcular numero medio de clientes
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    # Si tengo pista libre, aterrizo
    if (pistasOcupadas < nPistas):
        # Actualizo variables contadores
        n1 += 1                     # El avión entra en n1
        pistasOcupadas+=1           # Y ocupa una pista
        NLL1 += 1                   # numero de llegadas al n1
        LL1.append(t_suc)
        t = t_suc
        
        Z = t_aterrizaje() #random.normalvariate(mu11, sigma11) #genero tiempo aterrizaje
        LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + Z
        
        print(f'Avion finalizará aterrizaje en el instante {t+Z}')
        print(f'Hay  {pistasOcupadas} pistas ocupadas')
    
    else:
        #si no tengo pista, no aterrizo, me pongo en cola aterrizaje
        esperando_a_aterrizar+=1
        print(f'Todas la pistas ocupadas. {esperando_a_aterrizar} aviones esperndo aterrizar')

    #=========================================================================#

    # Genero tiempo siguente llegada -> esto tiene sentido hacerlo aqui?
    Y = t_llegada(t)
    print(f'ESTAMOS EN EL ISNTANTE {t}, queda {Y} para otro vuelo')
    # Compruebo si el siguiente avión llega antes de T
    if (t + Y < T):
        LISTA['tLL1'] = t + Y #añado otro tiempo de llegada
        print(f'El siguiente avion llegará en el instante {t+Y}')
    
    else:
        print(f'Horizonte alcanzado {t + Y} , {T}, no llegan más aviones')

#### rutina traslado

In [97]:
def rutina_traslado(t_suc): 
    # un coche recoge al avión y lo desaloja de la pista (salida de N1)
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_aterrizar, esperando_a_despegar
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, esperando_coche, pistasOcupadas,n_cola_aterrizaje, nPista
    global esperando_puerta
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    t=t_suc #Actualizamos tº de simulación
    
    n1 -=1              # El avión sale del nodo 1
    pistasOcupadas-=1   # Y despeja una pista
    
    # si hay coches disponibles -> traslado
    if coches_ocupados < nCoches:
        print(f'---Ahora hay {pistasOcupadas} pistas ocupadas')
        
        n2 +=1                  # El avion entra en n2
        coches_ocupados +=1     # Y ocupa un coche
        
        Z = t_traslado()
        print(f'----Comienza el traslado que durará {Z}----')
        print(f'----El avión llegará a la zona de desembarque en el instante {t + Z}----')
        print(f'----Ahora hay {nCoches - coches_ocupados} coches disponibles')
        LISTA['tS2'][LISTA['tS2'].index(10E9)] = t + Z # tiempo de salida del N2, que se corresponde con el final del traslado
        
    else: #si no hay coche disponibles 
        esperando_coche +=1 #lo pongo a la cola de ser trasladado
        print('---no hay vehículos disponibles. Hay {esperando_coche} aviones esperando a ser trasladados---')

    if esperando_a_aterrizar > 0: # ahora que se han liberado pistas, atendemos a la cola de aterrizaje
        Y = t_aterrizaje()                              # genero tiempo que tardará en aterrizar
        LISTA['tS1'][LISTA['tS1'].index(10E9)] = t + Y  # registro el tiempo en el que saldrá el avión del nodo 1
        
        n1+=1                       # El avión en cola entra al n1,
        pistasOcupadas += 1         # Ocupa una pista,
        esperando_a_aterrizar -= 1  # y libera su puesto en la cola.
        
    elif esperando_a_despegar > 0:
        W = t_salidas()                                 # genero tiempo que tardará en despegar
        LISTA['tSS'][LISTA['tSS'].index(10E9)] = t + W  # registro el tiempo en el que saldrá el avión del nodo 1

        n1+=1                       # El avión en cola entra al n1,
        pistasOcupadas +=1          # Ocupa una pista,
        esperando_a_despegar -=1    # Y libera su puesto en la cola.
        

#### rutina desembarque

In [98]:
def rutina_desembarque(t_suc): 
    # avion en aeropuerto: desembarcan pasajeros, embarcan los nuevos y traslado a pistas
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_despegar, puertas_ocupadas
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, esperando_coche, pistasOcupadas,n_cola_aterrizaje, nPista
    global esperando_a_aterrizar, esperando_puerta
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    t=t_suc #Actualizamos tº de simulación
    
    n2 -=1                  # El avión sale de n2
    coches_ocupados -=1     # Y libera su vehículo
    
    
    #si hay puertas de embarque disponibles
    if puertas_ocupadas < nPuertas:

        n3 +=1                  # El avión entra en n3
        puertas_ocupadas +=1    # Y ocupa una puerta
        
        Z = t_desembarques()
        print(f'----Comienza el cambio de pasajeros y traslado a zona de maniobra que durará {Z}----')
        print(f'----Ahora hay {nPuertas - puertas_ocupadas} puertas disponibles')
        LISTA['tS3'][LISTA['tS3'].index(10E9)] = t + Z
        print(f'---El avion finalizará el embarque en {Z} mins, en el instante {t + Z}')
        
    else: #si no hay puertas disponibles 
        esperando_puerta+=1 #esperando puerta embarque
        print('---No hay puertas de embarque disponibles.---')
        print('---Hay {esperando_puerta} aviones esperando a ser asignados puerta---')

    # si tengo aviones pendientes de ser trasladados
    if esperando_coche > 0: 

        n2+=1                       # El avión en cola entra al n2
        coches_ocupados += 1        # Ocupa un coche,
        esperando_coche -= 1        # Y libera su puesto en la cola.
        
        Y = t_traslado()
        LISTA['tS2'][LISTA['tS2'].index(10E9)] = t + Y

        print(f'---Un avión en cola es trasladado.---')
        print(f'---El avión llegará a terminal en el instante {t + Y}')
        print(f'---Quedan {esperando_coche} aviones pendientes de traslado---')       
        
    


#### rutina despegue (vuelta al nodo 1)

In [99]:
def rutina_despegue(t_suc): 
    # sale del nodo 3 y espera a poder salir del sistema (nodo SS)    
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_despegar, puertas_ocupadas
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, esperando_coche, pistasOcupadas,n_cola_aterrizaje, nPista
    global esperando_puerta, nSS, n4, esperando_a_aterrizar
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    t=t_suc #Actualizamos tº de simulación
    
    n3 -=1                  # El avión sale del n3
    puertas_ocupadas -=1    # Y libera un puerta
    
    # Si hay pistas libres
    if (pistasOcupadas < nPistas):
        
        n1+=1               # El avión entra al n1
        pistasOcupadas+=1   # Y ocupa una pista

        # generamos tiempo de despegue
        Z = t_salidas()
        LISTA['tSS'][LISTA['tSS'].index(10E9)] = t + Z

        print(f'----Un avión va a despegar.----')
        print(f'----Hay {pistasOcupadas} pistas ocupadas---')
        print(f'----El avion finalizará el despegue en el instante {t + Z}')

    else:    
        esperando_a_despegar+=1 #esperando para el despegue
        print('----No hay pistas disponibles.---')
        print('----Hay {esperando_a_despegar} aviones en cola de despegue---')
        
    # Manejo la cola del nodo anterior
    # si quedan aviones esperando puerta
    if esperando_puerta > 0:
        n3+=1                   # El avion entra al n3
        puertas_ocupadas += 1   # ocupa una puerta
        esperando_puerta-=1     # Y libera su puesto en la cola

        # Genero tiempo de desembarque
        Y = t_desembarques()
        LISTA['tS3'][LISTA['tS3'].index(10E9)] = t + Y

        print(f'----Un avión en cola ha sido asignado puerta.')
        print(f'----El avión finalizará el desembarque en el instante {t + Y}')
        print(f'----Quedan {esperando_puerta} aviones esperando ser asignados una puerta')
        
    

avión sale del sistema

In [100]:
def rutina_salida_sistema(t_suc): 
    # sale del nodo 3 y espera a poder salir del sistema (nodo SS)    
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_despegar, puertas_ocupadas
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, esperando_coche, pistasOcupadas,n_cola_aterrizaje, nPista
    global esperando_puerta, nSS, n4, esperando_a_aterrizar
    
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)

    t=t_suc #Actualizamos tº de simulación

    n1 -=1              # El avión sale del nodo 1
    pistasOcupadas-=1   # Y despeja una pista
    
    # Manejo la cola del nodo anterior
    # si quedan aviones esperando a despegar, y pistas disponibles para ello
    if esperando_a_despegar > 0:
        
        n1+=1                       # El avión en cola entra al n1,
        pistasOcupadas +=1          # Ocupa una pista,
        esperando_a_despegar -=1    # Y libera su puesto en la cola.
        
        Y = t_salidas()
        LISTA['tSS'][LISTA['tSS'].index(10E9)] = t + Y


        print(f'---Tengo {esperando_a_despegar} aviones pendientes de despegar')
        print(f'---Programamos la salida del avión que será en el instante {t + Y}')
        print(f'{esperando_a_despegar} aviones en espera y {nPistas - pistasOcupadas} pistas disponibles')
        

# MAIN

In [101]:

def simul_main():
    global t, n1, n2, n3, NLL1, LL1, LISTA, coches_ocupados, esperando_a_despegar, esperando_a_aterrizar, puertas_ocupadas
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, esperando_coche, pistasOcupadas,n_cola_aterrizaje, nPistas
    global esperando_a_aterrizar, esperando_puerta
    
    t = 0                   # Variable reloj. tiempo asíncrono.
    n1 = n2 = n3 = 0        # Variables de estado nº clientes
    NLL1 = NLL2 = NLL3 = 0  # Variables contador nº llegadas
    NS1 = NS2 = NS3 = 0     # Variables contador nº salidas
   
    # Arrays for arrival and departure times
    LL1 = []
    LL2 = []
    LL3 = []
    S1 = []
    S2 = []
    S3 = []

    n_med_n1 = n_med_n2 = n_med_n3 = 0 #nº medio de pistas ocupadas
    nmed_pistasOcupadas = 0
    coches_ocupados = 0
    puertas_ocupadas = 0

    n_cola_aterrizaje = 0 # Contador de aviones esperando para aterrizar
    n_cola_traslado = 0
    pistasOcupadas = 0
    esperando_a_despegar = 0
    esperando_a_aterrizar = 0

    esperando_coche = 0
    esperando_puerta = 0


    X = t_llegada(0)
    rutina_aterrizaje(X)
    while (t <= T and (LISTA['tLL1'] != 10E9 or LISTA['tS1'] != 10E9 or LISTA['tS2'] != 10E9 or LISTA['tS3'] != 10E9 or LISTA['tSS'] != 10E9)):
        print(LISTA)
        min_key = min(
            LISTA.keys(), 
            key=lambda k: min(LISTA[k]) if isinstance(LISTA[k], list) else LISTA[k]
        )
        if(min_key=='tLL1'): # Si el próximo evento es un aterrizaje (LLN1)
            print('----Ahora toca una llegada----')
            t_suc=LISTA['tLL1']
            LISTA['tLL1']= 10E9
            rutina_aterrizaje(t_suc)
            
            print(LISTA)
            print()
            
        elif(min_key=='tS1'): # Si el próximo evento es un traslado (SN1)
            print('----Ahora toca un traslado----')
            t_suc=min(LISTA['tS1'])
            LISTA['tS1'][LISTA['tS1'].index(min(LISTA['tS1']))] = 10E9
            rutina_traslado(t_suc)
            
            print(LISTA)
            print()
            
        elif(min_key=='tS2'): # Si el próximo evento es un deembarque (SN2)
            print('----Ahora toca un cambio de pasajeros----')
            t_suc=min(LISTA['tS2'])
            LISTA['tS2'][LISTA['tS2'].index(min(LISTA['tS2']))] = 10E9
            rutina_desembarque(t_suc)
            
            print(LISTA)
            print()
            
        elif(min_key=='tS3'): # Si el próximo evento es un deembarque (SN3)
            print('----Ahora toca esperar a torre de control----')
            t_suc=min(LISTA['tS3'])
            LISTA['tS3'][LISTA['tS3'].index(min(LISTA['tS3']))] = 10E9
            rutina_despegue(t_suc)
            
            print(LISTA)
            print()
            
        elif(min_key=='tSS'): # Si el próximo evento es un deembarque (SN4)
            print('----Ahora toca esperar a torre de control----')
            t_suc=min(LISTA['tSS'])
            LISTA['tSS'][LISTA['tSS'].index(min(LISTA['tSS']))] = 10E9
            rutina_salida_sistema(t_suc)
            
            print(LISTA)
            print()
    

In [102]:
simul_main()

----Llega un vuelo en el minuto 13.60304288389353----
Avion finalizará aterrizaje en el instante 27.557912970845415
Hay  1 pistas ocupadas
ESTAMOS EN EL ISNTANTE 13.60304288389353, queda 0.719112762622178 para otro vuelo
El siguiente avion llegará en el instante 14.322155646515707
{'tLL1': 14.322155646515707, 'tS1': [27.557912970845415, 10000000000.0, 10000000000.0], 'tS2': [10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0], 'tS3': [10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0, 10000000000.0

In [103]:
print(n1)
print(n2)
print(n3)
print(pistasOcupadas)
print(coches_ocupados)
print(puertas_ocupadas)


2
0
0
2
0
0


In [104]:
print(n_med_n1)
print(n_med_n2)
print(n_med_n3)
print(nmed_pistasOcupadas)

4863.751003545324
414.9011094688781
2283.80927141882
4863.751003545324
